In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Resolving Conflicts Using *Precedence Declarations*

This file shows how *shift/reduce* and *reduce/reduce* conflicts can be resolved using *operator precedence declarations*.

## Specification of the Scanner

We implement a minimal scanner for arithmetic expressions.

In [ ]:
import ply.lex as lex

tokens = [ 'NUMBER' ]

def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = float(t.value)
    return t

literals = ['+', '-', '*', '/', '^', '(', ')']

t_ignore  = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `expr`, but we dont't have to specify that.  The default
start variable is the first vvariable that is defined.

In [ ]:
start = 'expr'

The following *operator precedence declarations* declare that the operators `+`and `-` have a lower precedence than the operators `*`and `/`.  Furthermore, they specify that all these operators are *left associative*.  Operators can also be declared as *right associative* using the keyword `right` or as *non-associative* using the keyword `nonassoc`.

In [ ]:
precedence = (
    ('left', '+', '-'),
    ('left', '*', '/'),
    ('right', '^')
)

Without precedence declarations, the grammar below is ambiguous.

In [ ]:
def p_expr(p):
    """
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | expr '^' expr
         | '(' expr ')'
         | NUMBER            
    """
    pass

We define `p_error` to prevent a warning.

In [ ]:
def p_error(t):
    pass

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Let's look at the action table that is generated.  Conflicts are always resolved in favour of shifting.

In [ ]:
!cat parser.out